In [ ]:
#@markdown # Setup the demo app

def changeDirectory(p):
  from os import chdir
  chdir(p)

def createDirectory(p):
    try:
        from os import makedirs
        makedirs(p)
        print(f"Directory '{p}' created")
    except OSError as e:
        print(f"Directory '{p}' already exists")

def mountGdriveIfNotMounted():
    if not exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
        print("Gdrive mounted")

def normalizePath(p, useGdrive=False):
    from os import makedirs
    if useGdrive:
        mountGdriveIfNotMounted()
        path = "/content/drive/MyDrive/" + p
    else:
        path = "/content/" + p
    createDirectory(path);
    return path;

def sys(c, logOutput = False):
    from subprocess import run, PIPE
    p = run(c, shell=True, stdout=PIPE, stderr=PIPE)
    output = p.stdout.decode()
    errors = p.stderr.decode()
    if logOutput :
        print(output)
    if errors != "" :
        print(errors)

def exists(p):
    from os import path
    return path.exists(p)

def isEmpty(p):
    from os import listdir
    return len(listdir(p)) == 0

def cloneRepo(repo, destination):
    changeDirectory("/content")
    command = f"git clone --recurse-submodules https://{repo}.git"
    if destination != "":
        sys(command  +" "+ destination)
    else:
        sys(command)
    print(f"Repo '{repo}' cloned to '{destination}'")
    changeDirectory("/content")

def updateRepo(repo, destination):
    changeDirectory(destination)
    sys("git pull --recurse-submodules")
    print(f"Repo {repo} in directory {destination} updated")
    changeDirectory("/content")

def clearDirectory(destination):
    sys(f"rm -f -r {destination}")
    print(f"Directory {destination} deleted")
    changeDirectory("/content")

def pipInstall(parameters):
    sys(f"pip install {parameters}")
    print(f"'pip install' completed with parameters '{parameters}'")

def loginToHuggingFace():
    from huggingface_hub import notebook_login
    sys('git config --global credential.helper store')
    notebook_login()

#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader
!nvidia-smi

#@markdown ### Repo

#@markdown Which repo do you want to use

REPO = "github.com/jslegers/stable-diffusion" #@param ["github.com/jslegers/stable-diffusion", "huggingface.co/spaces/johnslegers/stable-diffusion"]

#@markdown Where to install

INSTALL_REPO_ON_GDRIVE = False #@param {type:"boolean"}

INSTALLATION_PATH = "stable-diffusion" #@param {type:"string"}
INSTALLATION_PATH = normalizePath(INSTALLATION_PATH, INSTALL_REPO_ON_GDRIVE)

IF_DIRECTORY_NOT_EMPTY = "Update" #@param ["Update", "Reinstall", "Do nothing"]

if exists(INSTALLATION_PATH) and not isEmpty(INSTALLATION_PATH):
    if IF_DIRECTORY_NOT_EMPTY == "Update":
        updateRepo(REPO, INSTALLATION_PATH)
    elif IF_DIRECTORY_NOT_EMPTY == "Reinstall":
        clearDirectory(INSTALLATION_PATH)
        cloneRepo(REPO, INSTALLATION_PATH)
else:
    cloneRepo(REPO, INSTALLATION_PATH)
#pipInstall(f"git+file:{INSTALLATION_PATH}")
pipInstall("-q huggingface_hub")

%cd /content/stable-diffusion
!pip install -q -r requirements.txt

#@markdown ### Login to HuggingFace

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights.

#@markdown Please, visit the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. You have to be a registered user in Hugging Face Hub, and you'll also need to use an access token for the code to work.

loginToHuggingFace()

In [ ]:
#@markdown # Run the demo app

%cd /content/stable-diffusion
!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python app.py